# This is an example of CHM evalution script writen in Jupyter Notebook

In [1]:
#Import dependent modules

import openpyxl
import os
from datetime import datetime,date

In [55]:
#Decalre functions

#parse the .xlsx report file and transform it into a dictionary structure
def parse_report(file_path):
    
    #find specific columns numbers  
    def locate_columns(sheet, targets=["Event Type", "Timestamp"], n_row=2):
        target_columns = {}
        for i in range(1, sheet.max_column+1):
            column_name = sheet.cell(row=n_row, column=i).value 
            if column_name in targets:
                target_columns[column_name] = i
        return target_columns
    
    #computer the time difference into seconds, negative vaule is provided
    def get_time_diff(time_a, time_b):
        date_time_a = datetime.combine(datetime.today(), time_a)
        date_time_b = datetime.combine(datetime.today(), time_b)
        time_format = '%H:%M:%S'
        time_diff = date_time_a - date_time_b
        return time_diff.total_seconds()
    
    
    #Load .xlsx file and locate prediction and ground_truth pages
    report = openpyxl.load_workbook(file_path)
    sheet_names = report.get_sheet_names()
    prediction_sheet = report.get_sheet_by_name(sheet_names[2])
    truth_sheet = report.get_sheet_by_name(sheet_names[0])
    
    #locate specfic columns
    prediction_columns = locate_columns(prediction_sheet)
    truth_columns = locate_columns(truth_sheet)
    
    #Insert result into the result dictionary
    result = {}
    for i in range(3,prediction_sheet.max_row + 1):
        if prediction_sheet.cell(row = i,column = 1).value is None:
            break
            
        else:
            #Compute the time difference
            test_time = prediction_sheet.cell(row = i,column = prediction_columns["Timestamp"]).value
            gt_time = truth_sheet.cell(row = i,column =  truth_columns["Timestamp"]).value
            time_diff = get_time_diff(test_time, gt_time)

            file_name = prediction_sheet.cell(row = i,column = 2).value
            truth_event = truth_sheet.cell(row = i,column = truth_columns["Event Type"]).value
            predicted_event = prediction_sheet.cell(row = i,column = prediction_columns["Event Type"]).value
            prediction_id = int(prediction_sheet.cell(row=i, column=1).value)
            result[prediction_id] = {"file_name":file_name, "time_diff":time_diff,\
                                    "truth":truth_event,"prediction": predicted_event}                                         
  
                                                
    return result

In [57]:
# Show case the functionality
if __name__ == "__main__":
    
    #parse the the .xlsx file into dictionary
    report_path = "Test.xlsx"
    result = parse_report(report_path)
    #show some results
    print(result[1])
    print(result[2])

{'file_name': u'ISprayPainting01', 'time_diff': 63.0, 'prediction': u'Blockage (1)', 'truth': u'Spray (2)'}
{'file_name': u'ISprayPainting02', 'time_diff': -4.0, 'prediction': u'Defocus (3)', 'truth': u'Spray (2)'}
